<table style="width: 100%; border-collapse: collapse;" border="0">
<tr>
<td><b>Created:</b> Monday 30 January 2017</td>
<td style="text-align: right;"><a href="https://www.github.com/rhyswhitley/fire_limitation">github.com/rhyswhitley/fire_limitation</td>
</tr>
</table>

<div>
<center>
<font face="Times">
<br>
<h1>Quantifying the uncertainity of a global fire limitation model using Bayesian inference</h1>
<h2>Part 1: Staging data for analysis</h2>
<br>
<br>
<sup>1,* </sup>Douglas Kelley, 
<sup>2 </sup>Ioannis Bistinas, 
<sup>3, 4 </sup>Chantelle Burton, 
<sup>1 </sup>Tobias Marthews, 
<sup>5 </sup>Rhys Whitley
<br>
<br>
<br>
<sup>1 </sup>Centre for Ecology and Hydrology, Maclean Building, Crowmarsh Gifford, Wallingford, Oxfordshire, United Kingdom
<br>
<sup>2 </sup>Vrije Universiteit Amsterdam, Faculty of Earth and Life Sciences, Amsterdam, Netherlands
<br>
<sup>3 </sup>Met Office United Kingdom, Exeter, United Kingdom
<br>
<sup>4 </sup>Geography, University of Exeter, Exeter, United Kingdom
<br>
<sup>5 </sup>Natural Perils Pricing, Commercial & Consumer Portfolio & Pricing, Suncorp Group, Sydney, Australia
<br>
<br>
<h3>Summary</h3>
<hr>
<p> 
This notebook aims to process the separate netCDF4 files for the model drivers (X<sub>i=1, 2, ... M</sub>) and model target (Y) into a unified tabular data frame, exported as a compressed comma separated value (CSV) file. This file is subsequently used in the Bayesian inference study that forms the second notebook in this experiment. The advantage of the pre-processing the data separately to the analysis allows for it be quickly staged on demand. Of course other file formats may be more advantageous for greater compression (e.g. SQLite3 database file).
</p>
<br>
<b>You will need to run this notebook to prepare the dataest before you attempt the Bayesian analysis in Part 2</b>.
<br>
<br>
<br>
<i>Python code and calculations below</i>
<br>
<hr>
</font>
</center>
</div>

## Load libraries

Changed

In [65]:
# data munging and analytical libraries 
import re
import os
import numpy as np
import pandas as pd
from netCDF4 import Dataset 

# graphical libraries
import matplotlib.pyplot as plt
%matplotlib inline

# set paths
inPath = "../outputs/amazon_region/"
outPath = "../data/amazon_inference_data"

satalites = ["TERRA_M__M", "TERRA_M__T", "AQUA_M__M", "AQUA_M__T"]

nmonths_import = 16*12

print (os.getcwd())
print (os.listdir(inPath))

/net/spice/scratch/cburton/ConFIRE/amazon_fires/optimise_run_model
['human', 'burnt_area_MCD64A1.006.nc', 'vegetation', 'burnt_area-GFED4s_2.5degree_2001-2016.nc', 'climate', 'climate.zip', 'fire_counts', 'OneDrive_1_2-20-2020.zip', 'OneDrive_1_2-20-2020.zip.xue9uiu.partial']


## Import and clean data

Set the directory path and look for all netcdf files that correspond to the model drivers and target.

In [66]:
driver_paths_all = [os.path.join(dp, f) for (dp, _, fn) in os.walk(inPath) for f in fn if f.endswith('.nc')]
print (driver_paths_all)

['../outputs/amazon_region/burnt_area_MCD64A1.006.nc', '../outputs/amazon_region/burnt_area-GFED4s_2.5degree_2001-2016.nc', '../outputs/amazon_region/human/dt-cropland2001-2019.nc', '../outputs/amazon_region/human/fract_agr2001-2019.nc', '../outputs/amazon_region/human/dt_cropland2001-2019.nc', '../outputs/amazon_region/human/cropland2001-2019.nc', '../outputs/amazon_region/human/population_density-PD_HYDEv3.2_2001-2018.nc', '../outputs/amazon_region/human/dt_pasture2001-2019.nc', '../outputs/amazon_region/human/dt-fract_agr2001-2019.nc', '../outputs/amazon_region/human/dt_fract_agr2001-2019.nc', '../outputs/amazon_region/human/dt-pasture2001-2019.nc', '../outputs/amazon_region/human/pasture2001-2019.nc', '../outputs/amazon_region/vegetation/vegcover-2001-June2018.nc', '../outputs/amazon_region/vegetation/treecover-2001-June2018.nc', '../outputs/amazon_region/vegetation/MaxOverMean_soilw.0-10cm.gauss.2000-2019.nc', '../outputs/amazon_region/vegetation/MaxOverMean_soilw.0-10cm.gauss.200

In [67]:
#Find files
driver_paths_all = [os.path.join(dp, f) for (dp, _, fn) in os.walk(inPath) for f in fn if f.endswith('.nc')]
#Take out if using burnt area:
#driver_paths_all = [path for path in driver_paths_all if path != '../outputs/amazon_region/burnt_area-GFED4s_2.5degree_2001-2016.nc']
driver_names_all = [re.search('^[a-zA-Z_]*', os.path.basename(fp)).group(0) for fp in driver_paths_all]

file_table = pd.DataFrame({'filepath': driver_paths_all, 'file_name': driver_names_all})

#Use 'fire' for burnt area, and fire_count for fire count:

def checkFilename(driver_name, sat):
    
    if "fire" in driver_name:
        if sat in driver_name: return True
        return False
    
    return True

def select_files_for_sat(sat):
    test = [checkFilename(driver_name, sat) for driver_name in driver_names_all]
    
    driver_paths = np.array(driver_paths_all)[test]
    driver_names = np.array(driver_names_all)[test]
    
    driver_names = ["fire" if "fire" in driver_name else driver_name for driver_name in driver_names]
    return driver_paths, driver_names
    
driver_info = [select_files_for_sat(sat) for sat in satalites]

Define a function to extract the variable values from each netCDF4 file. Variables are flattened from a 3 dimensional array to 1 dimensional version, pooling all values both spatially and temporily. 

Don't know if this is the correct way to do this, but will come back to it once I understand the model (and its optimisation) better.

In [68]:
from pdb import set_trace as browser

def nc_extract(fpath):
    print("Processing: {0}".format(fpath))
    with Dataset(fpath, 'r') as nc_file:
        gdata = np.array(nc_file.variables['variable'][:,:,:])
        if (len(gdata) < nmonths_import): return None
        gdata = gdata[13:nmonths_import]
        gdata[gdata < -9E9] = np.nan
        gflat = gdata.flatten()
        if type(gdata) == np.ma.core.MaskedArray:
            return gflat[~gflat.mask].data
        else:
            return gflat.data

Execute the above function on all netCDF4 file paths.

In [69]:
values = [ [nc_extract(dp) for dp in driver_paths[0]] for driver_paths in driver_info]
print ('Hello')
print (driver_info)

Processing: ../outputs/amazon_region/burnt_area_MCD64A1.006.nc
Processing: ../outputs/amazon_region/burnt_area-GFED4s_2.5degree_2001-2016.nc
Processing: ../outputs/amazon_region/human/dt-cropland2001-2019.nc
Processing: ../outputs/amazon_region/human/fract_agr2001-2019.nc
Processing: ../outputs/amazon_region/human/dt_cropland2001-2019.nc
Processing: ../outputs/amazon_region/human/cropland2001-2019.nc
Processing: ../outputs/amazon_region/human/population_density-PD_HYDEv3.2_2001-2018.nc
Processing: ../outputs/amazon_region/human/dt_pasture2001-2019.nc
Processing: ../outputs/amazon_region/human/dt-fract_agr2001-2019.nc
Processing: ../outputs/amazon_region/human/dt_fract_agr2001-2019.nc
Processing: ../outputs/amazon_region/human/dt-pasture2001-2019.nc
Processing: ../outputs/amazon_region/human/pasture2001-2019.nc
Processing: ../outputs/amazon_region/vegetation/vegcover-2001-June2018.nc
Processing: ../outputs/amazon_region/vegetation/treecover-2001-June2018.nc
Processing: ../outputs/amazon

Turn this into a dataframe for the analysis.

In [73]:
# turn list into a dataframe
def makeDataFrave(value, driver_names):
    df = pd.DataFrame(np.array(value).T, columns=driver_names)
    print(df.info())
    df.dropna(inplace=True)
    return df
fire_df = [makeDataFrave(value, driver_names[1]) for value, driver_names in zip(values, driver_info)]
#fire_df.info()
# replace null flags with pandas null
#fire_df.replace(fire_df < -3e38, np.nan, inplace=True)
#fire_df[] = np.nan
# drop all null rows (are ocean and not needed in optim)
#fire_df.dropna(inplace=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114202 entries, 0 to 114201
Data columns (total 26 columns):
burnt_area_MCD        44750 non-null float32
burnt_area            44750 non-null float32
dt                    44750 non-null float32
fract_agr             44750 non-null float32
dt_cropland           44750 non-null float32
cropland              44750 non-null float32
population_density    44750 non-null float32
dt_pasture            44750 non-null float32
dt                    44750 non-null float32
dt_fract_agr          44750 non-null float32
dt                    44750 non-null float32
pasture               44750 non-null float32
vegcover              44750 non-null float32
treecover             44750 non-null float32
MaxOverMean_soilw     44750 non-null float32
MaxOverMean_soilw     44750 non-null float32
nonetreecover         44750 non-null float32
relative_humidity     44750 non-null float32
lightning             44750 non-null float32
emc                   44750 non-nu

Check that we've built it correctly.

In [74]:
for df in fire_df:
    print(df.head(10))

    burnt_area_MCD  burnt_area        dt  fract_agr  dt_cropland  cropland  \
25        0.000219    0.000792  0.000075   0.586741     0.000075  0.075360   
26        0.000861    0.001852  0.000132   0.447438     0.000132  0.084143   
27        0.000117    0.003716 -0.000030   0.220080    -0.000030  0.097023   
28        0.000354    0.005329 -0.000024   0.196809    -0.000024  0.062484   
29        0.000105    0.003184 -0.000038   0.165909    -0.000038  0.044306   
46        0.000000    0.000022  0.000050   0.070869     0.000050  0.017306   
47        0.000488    0.000675  0.000166   0.324843     0.000166  0.049049   
48        0.000235    0.004220  0.000172   0.473489     0.000172  0.057812   
49        0.028126    0.017781  0.000101   0.229776     0.000101  0.066443   
50        0.029645    0.023555  0.000029   0.155881     0.000029  0.041235   

    population_density  dt_pasture        dt  dt_fract_agr  ...  \
25           55.566036    0.005015  0.005090      0.005090  ...   
26     

Export this to disk to be used by the analysis notebook - used gzip compression to save on space. Beware, because of there are approximation 10 million rows of data, this may take some time.

In [75]:
for sat, df in zip(satalites, fire_df):
    outPathi = outPath + '-' + sat + '.csv'
    savepath = os.path.expanduser(outPathi)
    df.to_csv(savepath, index=False)

<div>
<br>
<br>
<br>
<center>
<font size="5">
<a style="font-weight: bold; size: 5" href="http://localhost:8888/notebooks/notebooks/bayesian_inference.ipynb">Part 2: click here</a>
</font>
</center>
</div>